<a href="https://colab.research.google.com/github/anamilanezi/ebac-dados/blob/main/projetos/em17-analise-exploratoria-python-II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Análise Exploratória de Dados de Logística I
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **1\. Contexto**

Neste problema, iremos utilizar os dados sintetizados de fontes públicas (IBGE, IPEA, etc.) que são representativos dos desafios que a startup Loggi enfrenta no dia a dia, como otimização das rotas de entrega, alocação de entregas nos veículos da frota com capacidade limitada, etc. O dado bruto é um arquivo do tipo JSON com uma lista de instâncias de entregas. Cada instância representa um conjunto de entregas que devem ser realizadas pelos veículos do hub regional.

Os dados estão organizados da seguinte forma:
 - **name**: uma `string` com o nome único da instância;
 - **region**: uma `string` com o nome único da região do **hub**;
 - **origin**: um `dict` com a latitude e longitude da região do **hub**;
 - **vehicle_capacity**: um `int` com a soma da capacidade de carga dos **veículos** do **hub**;
 - **deliveries**: uma `list` de `dict` com as **entregas** que devem ser realizadas:
    - **id**: uma `string` com o id único da **entrega**;
    - **point**: um `dict` com a latitude e longitude da **entrega**;
    - **size**: um `int` com o tamanho ou a carga que a **entrega** ocupa no **veículo**.


# **2\. Pacotes e bibliotecas**

In [ ]:
import json
import pandas as pd
from pprint import pprint

# **3\. Exploração de dados**

## **3.1. Coleta**

Obtendo os dados brutos no formato `.json`:

In [ ]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json 

In [ ]:
with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

In [ ]:
# Formato do dado coletado:
type(data)

list

In [ ]:
# Quantidade de itens da lista
len(data)

199

In [ ]:
# Atribuindo o primeiro item da lista a uma variável para verificar sua organização
primeiro_item = data[0]
pprint(primeiro_item, depth=1)

{'deliveries': [...],
 'name': 'cvrp-2-df-33',
 'origin': {...},
 'region': 'df-2',
 'vehicle_capacity': 180}


In [ ]:
def print_dic(itemsDict, leftWidth, rightWidth, values=True):
  print(' TIPOS DE DADOS '.center(leftWidth + rightWidth, '-'))
  if values:
    for k, v in itemsDict.items():
      print(f"'{k}'".ljust(leftWidth, '.') + f'{v}'.ljust(rightWidth))
  else:
    for k, v in itemsDict.items():
      print(f"'{k}'".ljust(leftWidth, '.') + f'{type(v)}'.ljust(rightWidth))

#### 👉 Cada item da lista corresponde a um dicionário aninhado, que possui os valores de diferentes formatos:

In [ ]:
print_dic(primeiro_item, 30, 20, values=False)

----------------- TIPOS DE DADOS -----------------
'name'........................<class 'str'>       
'region'......................<class 'str'>       
'origin'......................<class 'dict'>      
'vehicle_capacity'............<class 'int'>       
'deliveries'..................<class 'list'>      


#### 👉 A chave `'deliveries'` tem como valor uma lista onde cada item é um dicionário

In [ ]:
# A lista possui 4588 dicionários
print(len(primeiro_item['deliveries']))

4588


In [ ]:
# Cada dicionário possui as chaves 'id', 'point' e 'size':
print_dic(primeiro_item['deliveries'][0], 20, 10)

------- TIPOS DE DADOS -------
'id'................313483a19d2f8d65cd5024c8d215cfbd
'point'.............{'lng': -48.11618888384239, 'lat': -15.848929154862294}
'size'..............9         


In [ ]:
print_dic(primeiro_item['deliveries'][0], 20, 10, values=False)

------- TIPOS DE DADOS -------
'id'................<class 'str'>
'point'.............<class 'dict'>
'size'..............<class 'int'>


## **3.2. Wrangling**

In [ ]:
# Converte dicionário em dataframe
deliveries_df = pd.DataFrame(data)

In [ ]:
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


In [ ]:
deliveries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              199 non-null    object 
 1   region            199 non-null    object 
 2   hub_lng           199 non-null    float64
 3   hub_lat           199 non-null    float64
 4   vehicle_capacity  199 non-null    int64  
 5   deliveries        199 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 9.5+ KB


As colunas `origin` e `deliveries` possuem dados em formato de dicionário e lista, respectivamente, e isso exige alguns métodos da biblioteca pandas para descompactar essas informações.

#### 👉 Iniciando com a coluna `origin`, vamos criar um segundo dataframe utilizando o método `.json_normalize`.

Na operação conhecia como `flatten`, transforma-se cada chave do JSON em uma nova coluna, separando as informações que estão aninhadas como dicionário mantendo o index original.

In [ ]:
hub_origin_df = pd.json_normalize(deliveries_df["origin"])
hub_origin_df.head()

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145


Com os dados separados em colunas, é possível juntar os dois dataframes com o método merge utilizando o index das linhas que foi mantido como chave para uní-los.

In [ ]:
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)

In [ ]:
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries,lng,lat
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po...",-48.054989,-15.838145
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p...",-48.054989,-15.838145
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p...",-47.893662,-15.805118
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p...",-48.054989,-15.838145


Com os dados unidos em um só dataframe, e as novas colunas de lng e lat, vamos remover a coluna `origin` contendo os dados no dicionário aninhado e reordenar as colunas.

In [ ]:
deliveries_df = deliveries_df.drop("origin", axis=1)
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
deliveries_df.head()

,name,region,lng,lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-48.054989,-15.838145,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-48.054989,-15.838145,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-47.893662,-15.805118,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-48.054989,-15.838145,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


Visto que em deliveries também existem informações referentes à latitude e longitude do ponto de entrega, vamos renomear as colunas recém criadas para especificar que se referem ao hub de origem:

In [ ]:
deliveries_df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"}, inplace=True)
deliveries_df.head()

,name,region,hub_lng,hub_lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-48.054989,-15.838145,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-48.054989,-15.838145,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-47.893662,-15.805118,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-48.054989,-15.838145,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


Agora vamos fazer o tratamento da coluna deliveries, onde cada linha contém uma lista de JSON (que corresponde ao formato de um dicionário python). 

#### 👉 A partir da coluna `deliveries` cria-se um novo dataframe, utilizando o método `explode`, que transforma cada elemento da lista em uma nova linha do dataframe. 

Verifica-se que o dataframe criado possui 636149 linhas, e elas mantém o index da linha do dataframe de origem.

In [ ]:
deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")
print(len(deliveries_exploded_df))

636149


In [ ]:
deliveries_exploded_df.head()

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."


In [ ]:
deliveries_exploded_df.tail()

,deliveries
198,"{'id': '21693bf442ac5890adbdf2648c12881a', 'po..."
198,"{'id': '7aaa35088b37b6e542c4cd69663a7ebf', 'po..."
198,"{'id': '60c00d5390da4f28167439cd9c566703', 'po..."
198,"{'id': '51f456963785e7381243ff7baf7efd06', 'po..."
198,"{'id': 'b7078c815198669e2aab4336e94c0bb8', 'po..."


#### 👉 Cada linha (dicionário/json) do dataframe dá origem a 3 novos dataframes, onde cada um representa uma informação de interesse contida ali dentro.

Agora cada linha será primeiramente transformada em um dataframe com o método apply sobre a coluna, selecionando as informações através da chave do dicionário das informações referentes à latitude, longitude e tamanho da carga (o id não será utilizado). 

In [ ]:
size_df = pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"})

point_lng_df = pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"})

point_lat_df = pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"})

In [ ]:
size_df.head()

,delivery_size
0,9
0,2
0,1
0,2
0,7


In [ ]:
point_lng_df.head()

,delivery_lng
0,-48.116189
0,-48.118195
0,-48.112483
0,-48.118023
0,-48.114898


In [ ]:
point_lat_df.head()

,delivery_lat
0,-15.848929
0,-15.850772
0,-15.847871
0,-15.846471
0,-15.858055


In [ ]:
# Uma lista contendo os DataFrames:
dfs = [size_df, point_lng_df, point_lat_df]

#### 👉 Com cada informação separada em um dataframe, eles são então concatenados em `deliveries_normalized_df`.

In [ ]:
deliveries_normalized_df = pd.concat(dfs, axis=1)

In [ ]:
deliveries_normalized_df.head()

,delivery_size,delivery_lng,delivery_lat
0,9,-48.116189,-15.848929
0,2,-48.118195,-15.850772
0,1,-48.112483,-15.847871
0,2,-48.118023,-15.846471
0,7,-48.114898,-15.858055


In [ ]:
# O mesmo processo:
deliveries_normalized_df = pd.concat([
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)
deliveries_normalized_df.head()

#### 👉 Os dados assim dispostos podem ser combinados com o dataframe inicial utilizando `merge`

Primeiro removemos a coluna `deliveries` visto que o novo dataframe `deliveries_normalized_df` foi criado a partir dos dados dela, e após isso as informações são mescladas utilizando o index das linhas. 



In [ ]:
deliveries_df = deliveries_df.drop("deliveries", axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)
deliveries_df.head()

,name,region,hub_lng,hub_lat,vehicle_capacity,delivery_size,delivery_lng,delivery_lat
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,9,-48.116189,-15.848929
1,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,2,-48.118195,-15.850772
2,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,1,-48.112483,-15.847871
3,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,2,-48.118023,-15.846471
4,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,7,-48.114898,-15.858055


## **3.3. Estrutura**

In [ ]:
# Número de linhas e colunas
deliveries_df.shape

(636149, 8)

In [ ]:
# Nome das colunas
deliveries_df.columns

Index(['name', 'region', 'hub_lng', 'hub_lat', 'vehicle_capacity',
       'delivery_size', 'delivery_lng', 'delivery_lat'],
      dtype='object')

In [ ]:
# Índex dos registros (linhas)
deliveries_df.index

RangeIndex(start=0, stop=636149, step=1)

In [ ]:
# Nome das colunas, número de registros e tipos de dados
deliveries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636149 entries, 0 to 636148
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              636149 non-null  object 
 1   region            636149 non-null  object 
 2   hub_lng           636149 non-null  float64
 3   hub_lat           636149 non-null  float64
 4   vehicle_capacity  636149 non-null  int64  
 5   delivery_size     636149 non-null  int64  
 6   delivery_lng      636149 non-null  float64
 7   delivery_lat      636149 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 38.8+ MB


## **3.4. Schema**


In [ ]:
# Sumário dos dados categóricos
deliveries_df.select_dtypes("object").describe().T

,count,unique,top,freq
name,636149,199,cvrp-1-df-87,5636
region,636149,3,df-1,304708


In [ ]:
# Sumário dos dados numéricos
deliveries_df.select_dtypes('int64').describe().transpose()

,count,mean,std,min,25%,50%,75%,max
vehicle_capacity,636149.0,180.000000,0.000000,180.0,180.0,180.0,180.0,180.0
delivery_size,636149.0,5.512111,2.874557,1.0,3.0,6.0,8.0,10.0


## **3.5. Dados faltantes**

Dados faltantes podem ser strings vazias (`""`), dados nulos (`None`), não disponíveis ou aplicáveis( `na`, `NA`, etc) e não numéricos (`nan`, `NaN`, etc). Podemos verificar quais colunas possuem dados faltantes utilizando `.isna()` e `.any()`

In [ ]:
deliveries_df.isna().any()

name                False
region              False
hub_lng             False
hub_lat             False
vehicle_capacity    False
delivery_size       False
delivery_lng        False
delivery_lat        False
dtype: bool

Caso fossem identificados dados faltantes, estes registros poderiam ser dispensados utilizando o método `df.dropna()`